In [51]:
import numpy as np
import pyomo.environ as pyo
from jobshop.params import JobShopRandomParams
from jobshop.mip.disjunctive import DisjModel
from jobshop.heurstic.grasp import simple_grasp
from jobshop.heurstic.evaluation import calc_makespan

Path-relinking is being included. Please take a look at [this section](#under-development-path-relinking)

In [52]:
params = JobShopRandomParams(10, 10, seed=12)

In [48]:
model = DisjModel(params)
solver = pyo.SolverFactory("cbc")
solver.options["cuts"] = "on"
solver.options["sec"] = 10

In [49]:
solver.solve(model, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  5 2022 

command line - C:\Users\Bruno\Documents\Programas\Cbc\bin\cbc.exe -cuts on -sec 10 -printingOptions all -import C:\Users\Bruno\AppData\Local\Temp\tmp_9w95dvd.pyomo.lp -stat=1 -solve -solu C:\Users\Bruno\AppData\Local\Temp\tmp_9w95dvd.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 10
Option for printingOptions changed from normal to all
Presolve 180 (-181) rows, 106 (-106) columns and 510 (-331) elements
Statistics for presolved model
Original problem has 180 integers (180 of which binary)
Presolved problem has 75 integers (75 of which binary)
==== 105 zero objective 2 different
105 variables have objective of 0
1 variables have objective of 1
==== absolute objective values 2 different
105 variables have objective of 0
1 variables have objective of 1
==== for integers 75 zero objective 1 different
75 variables have objective of 0
==== for integers absolute objective values 1 different
75 varia

{'Problem': [{'Name': 'unknown', 'Lower bound': 77.285439, 'Upper bound': 98.0, 'Number of objectives': 1, 'Number of constraints': 180, 'Number of variables': 106, 'Number of binary variables': 180, 'Number of integer variables': 180, 'Number of nonzeros': 1, 'Sense': 'minimize'}], 'Solver': [{'Status': 'aborted', 'User time': -1.0, 'System time': 10.02, 'Wallclock time': 10.02, 'Termination condition': 'maxTimeLimit', 'Termination message': 'Optimization terminated because the time expended exceeded the value specified in the seconds parameter.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 2124, 'Number of created subproblems': 2124}, 'Black box': {'Number of iterations': 47702}}, 'Error rc': 0, 'Time': 10.454984903335571}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [50]:
print(model.obj())

98.0


In [53]:
sol_grasp = simple_grasp(params, n_iter=1000, alpha=0.8, seed=12)

In [7]:
sol_grasp_alt = simple_grasp(params, n_iter=10, alpha=0.7, seed=15)

In [54]:
print(sol_grasp.value)

167


In [9]:
print(sol_grasp_alt.value)

78


In [10]:
graph = sol_grasp.graph
graph_alt = sol_grasp_alt.graph

In [11]:
calc_makespan(graph)

78

In [12]:
calc_makespan(graph_alt)

78

## Under development Path-relinking

In [14]:
def try_swap(graph, m, i, q):
    new_graph = graph.copy()
    new_graph.M[m].jobs.swap(i, q)
    calc_makespan(new_graph)
    if new_graph.C < graph.C:
        print("We did it, Adrian!")
    else:
        print("Failed")
    print(new_graph.C)
    return new_graph

In [15]:
graph.M

{1: {'key': 1, 'jobs': [5.0, 2.0, 1.0, 4.0, 3.0]},
 2: {'key': 2, 'jobs': [3.0, 5.0, 1.0, 2.0, 4.0]},
 3: {'key': 3, 'jobs': [5.0, 3.0, 2.0, 1.0, 4.0]},
 4: {'key': 4, 'jobs': [5.0, 2.0, 4.0, 1.0, 3.0]}}

In [16]:
graph_alt.M

{1: {'key': 1, 'jobs': [5.0, 2.0, 1.0, 4.0, 3.0]},
 2: {'key': 2, 'jobs': [3.0, 1.0, 5.0, 2.0, 4.0]},
 3: {'key': 3, 'jobs': [5.0, 2.0, 3.0, 1.0, 4.0]},
 4: {'key': 4, 'jobs': [5.0, 2.0, 4.0, 1.0, 3.0]}}

In [19]:
new_graph = try_swap(graph, 2, 5., 1.)
new_graph.M

Failed
78


{1: {'key': 1, 'jobs': [5.0, 2.0, 1.0, 4.0, 3.0]},
 2: {'key': 2, 'jobs': [3.0, 1.0, 5.0, 2.0, 4.0]},
 3: {'key': 3, 'jobs': [5.0, 3.0, 2.0, 1.0, 4.0]},
 4: {'key': 4, 'jobs': [5.0, 2.0, 4.0, 1.0, 3.0]}}